In [512]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [513]:
import pandas as pd

train = pd.read_csv('train_record.csv')
test = pd.read_csv('test_record.csv')

In [514]:
train

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
243,62.0,0,61,1,38,1,155000.00,1.1,143,1,1,270,0
244,55.0,0,1820,0,38,0,270000.00,1.2,139,0,0,271,0
245,45.0,0,2060,1,60,0,742000.00,0.8,138,0,0,278,0
246,45.0,0,2413,0,38,0,140000.00,1.4,140,1,1,280,0


In [515]:
x_train=train.drop(columns=['DEATH_EVENT'])
y_train=train['DEATH_EVENT']
y_train=y_train.to_numpy()

In [516]:
x_test=test.drop(columns=['DEATH_EVENT'])
y_test=test['DEATH_EVENT']

In [517]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [518]:
class ANN(object):
    def __init__(self):
        self.input_size = 12
        self.output_size = 1
        self.hidden_size = 6

        np.random.seed(50) 
        self.w1 = np.random.randn(self.input_size, self.hidden_size) 
        self.w2 = np.random.randn(self.hidden_size, self.output_size) 

    def _sigmoid(self, z):
        return 1/(1+np.exp(-z))
    
    def _sigmoid_gradient(self, z):
        z = 1/(1+np.exp(-z))
        return z*(1-(z))

    def _forward(self, X):
        a2=self._sigmoid(np.dot(self.w1.T, X.T))
        a3 = self._sigmoid(np.dot(self.w2.T, a2))
        return a3

    def _backward_gradient(self, X, y, learning_rate):
        m = X.shape[0]
        Z1 = np.dot(X, self.w1)
        A1 = self._sigmoid(Z1)

        Z2 = np.dot(A1, self.w2)
        A2 = self._sigmoid(Z2)

        E1 = A2 - y
        dW1 = np.dot(E1, self._sigmoid_gradient(Z2))

        E2= np.multiply(dW1, self.w2.T)
        dW2 = np.multiply(E2, self._sigmoid_gradient(Z1))

        W2_update = np.dot(A1.T, dW1) / m
        W1_update = np.dot(X.T, dW2) / m
    
        self.w2 = self.w2 - learning_rate * W2_update
        self.w1 = self.w1 - learning_rate * W1_update
       
    def train(self, X, y, iteration=25):
        for i in range(iteration):
            self._backward_gradient(X,y,0.005)

    def predict(self, X):    
        predictions = self._forward(X)
        predictions = [1 if i[0] >= 0.5 else 0 for i in predictions.T]
        return np.array(predictions)
    
    def score(self, predict, y):
        cnt = np.sum(predict==y)
        return (cnt/len(y))*100

In [519]:
model = ANN() 
model.train(x_train, y_train) 
pre_y = model.predict(x_test) 
score = model.score(pre_y, y_test)
print('predcit: ', pre_y)
print('anser:', y_test.ravel())
print('score: ', score)

predcit:  [1 1 1 0 0 0 1 1 1 1 0 1 0 0 1 0 0 1 0 1 1 0 0 1 0 0 1 1 0 0 0 0 0 0 0 0 1
 0 0 0 0 1 0 0 0 1 0 0 0 0 0]
anser: [1 1 1 1 1 1 1 1 0 0 1 1 1 1 0 1 0 1 1 1 1 1 0 0 1 0 1 1 0 0 0 0 0 0 1 0 1
 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
score:  68.62745098039215
